In [ ]:
import chess
from IPython.core.display import HTML
with open('style.html', 'r') as file:
     css = file.read()
HTML(css)

In [ ]:
%run Util/00_imports.ipynb
%run Util/01_functions_restructure.ipynb

In [ ]:
# Global variables:

USED_BOARDS = set()
SWAPS = {
        "vertical" : {x:x^56 for x in range(64)},
        "horizontal" : {x:x^7 for x in range(64)},
        "rotate_right" : {x:(((x >> 3) | (x << 3)) & 63) ^ 56 for x in range(64)},
        "rotate_180" : {x : x ^ 63 for x in range(64)},
        "rotate_left" : {x : (((x >> 3) | (x << 3)) & 63) ^ 7 for x in range(64)},
        "diagonal" : {x : ((x >> 3) | (x << 3)) & 63 for x in range(64)},
        "anti_diagonal" : {x : (((x >> 3) | (x << 3)) & 63) ^ 63 for x in range(64)}
    }
PIECE_LIST = [chess.Piece.from_symbol("K"), chess.Piece.from_symbol("k"), chess.Piece.from_symbol("R")]
USER_WANTS_PAWN = chess.Piece.from_symbol("P") in PIECE_LIST

Definition von Enum (Fand das Stroetmann nicht unnötig? Sollten wir net einfach üüber Dict iterieren?)

In [ ]:
class Swap_Type(Enum):
    VERTICAL = "vertical"
    HORIZONTAL = "horizontal"
    ROTATE_RIGHT = "rotate_right"
    ROTATE_180 = "rotate_180"
    ROTATE_LEFT = "rotate_left"
    DIAGONAL = "diagonal"
    ANTI_DIAGONAL = "anti_diagonal"

# Überlegung im Vergleich zu vorherigen Version:
* Muss der Nutzer wirklich einen Namen der Datei angeben? Können wir diesen nicht generisch erstellen?

# Voraussetzungen:
* Nutzer ist immer weiß

# Strukturüberlegung:
* Zentrale used_boards -> Müssen nicht übergeben werden
* Uniques werden gespeichert anstatt kompletter s_n Menge
* Angabe des Nutzers der aktuellen Situation
* Nutzer startet Berechnung:

# Benötigte Funktionen:

* Spiegelungen aus Uniques generieren; gen_mirroring(uniques):
    * Über Uniques iterieren
        * Über Spiegelarten iterieren
            * Spiegelung durchführen (ohne Objekterstellung in Integer-Schreibweise spiegeln)
            * Spiegelungen in Menge zusammenfassen
    * Menge zurückgeben

Menge an Schachbrettern in alle Richtungen spiegeln (falls überhaupt benötigt)

In [ ]:
def gen_mirroring(uniques):
    result = uniques.copy()
    for unique in uniques:
        result |= mirror_board_all(unique)
    return result

Schachbrett in alle Richtungen spiegeln

In [ ]:
# Implementierung ohne ENUM:
def mirror_board_all(board_int):
    result = set()
    for name, d in SWAPS.items():
        result.add(mirror_board(board_int, d))
    return result

In [ ]:
# Alternative Implementierung mit ENUM:
def mirror_board_all_enum(board_int):
    result = set()
    for sw_type in Swap_Type:
        result.add(mirror_board(board_int, SWAPS[sw_type.value]))
    return result

Mirror Board with bits in one direction

In [ ]:
# Mirroring mit 6 Bit Darstellung

def mirror_board(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in list -> Überlegung Liste global zu machen
    rounds = len(PIECE_LIST)
    # Remove turn
    board_int = board_int >> 1
    for index in range(0, rounds):
        result = (mirror[board_int & 63] << (6 * index) + 1) | result
        board_int = board_int >> 6
    return result

In [ ]:
'''
board = chess.Board(None)
map = {20: chess.Piece.from_symbol("Q"), 5: chess.Piece.from_symbol("k") }
board.set_piece_map(map)
display(board)
int_board = to_integer(board, plist)
print(int_board)
for i in mirror_board_all(int_board):
    print(str(i) + " Spiegelung:")
    display(to_board(i, plist))
'''

In [ ]:
# TODO: Erstellen nach Lukas seiner Bitmethode, wenn ich sie komplett
# Mirroring mit 7 Bit Darstellung
def mirror_board_7(board_int, mirror : dict):
    # Save turn
    result = board_int & 1
    # Get count of pieces saved in int
    rounds = int(math.log2(board_int) // 6)
    # Remove turn
    board_int = board_int >> 1
    for index in range(rounds, 0, -1):
        result = (mirror[board_int & 127] << (7 * (index-1)) + 1) | result
        board_int = board_int >> 7
    return result

* Start der Berechnung; start_calculation():
    * Bekommt Positionen der Figuren übergeben
    * Berechnung der ersten Menge S_0, damit auch Berechnung der ersten Uniques -> Rückgabe nur Uniques
        * Hierbei erste wichtige Frage: Werden in used_boards dann ebenfalls nur die Uniques gespeichert oder auch alle?
        * Schreiben wir die ganze S_0 Menge direkt in die Datei nach der Berechnung?
    * Schleife iterieren mit altbekannter Bedingung (wenn neue Menge leer oder sich nicht unterscheidet)
        * Berechnung nächster Menge mit Übergabe der Uniques vorheriger Menge

In [ ]:
def start_calculation():
    sn = calculate_s0()
    # while len(sn) != 0:
    #     sn = calculate_next_sn(sn)
    #     store_sn(sn)
    print("Done")

## Erstellung von S_0
> TODO: Einleitung schreiben

> TODO: Beschreibung von calculate_s0() schreiben

In [ ]:
def calculate_s0():
    # TODO: Hier muss noch irgendwo eine Berücksichtigug rein, dass der Bauer durch die Queen beim Erstellen von s0  ersetzt wird, dies muss dazu noch rückgängi gemacht werden, wenn die Queen durch den Pawn ersetzt wird.


    s0 = set()
    empty_board = chess.Board(None)
    # Turn has to be black while checkmate
    empty_board.turn = chess.BLACK

    # Create temp_piece_list for conversion between board_object and integer
    tmp_piece_list = []
    s0.add(to_integer(empty_board, tmp_piece_list))

    for piece in PIECE_LIST:
        s0 = fill_boards_with_piece(s0, piece, tmp_piece_list)
        tmp_piece_list.append(piece)
        print("Length s0:" + str(len(s0)))
        print("Actual size: " + str(len(gen_mirroring(s0))))
    store_sn(s0)
    return s0

> TODO: Beschreibung von fill_boards_with_piece() schreiben

In [ ]:
def fill_boards_with_piece(int_boards_set, piece, cur_piece_list):
    finished_boards = set()
    all_squares = set(range(64))
    piece_count = len(PIECE_LIST)
    for int_board in int_boards_set:
        board_o = to_board(int_board, cur_piece_list)
        used_squares = set(board_o.piece_map().keys())
        for square in all_squares:
            if square not in used_squares:
                tmp_piece_list = cur_piece_list.copy()
                board_object = board_o.copy()
                board_object.set_piece_at(square, piece)
                tmp_piece_list.append(piece)
                board_int = to_integer(board_object, tmp_piece_list)
                if len(used_squares) > 1 and not board_object.is_valid():
                    # Don't process invalid boards further
                    # than the second king
                    continue

                if board_object.is_checkmate():
                    if board_object.is_valid() and board_int not in USED_BOARDS:
                        finished_boards |= add_all_to_used(board_int)
                        continue

                if len(used_squares) + 1 < piece_count: #Board is valid, but needs more pieces
                    finished_boards.add(board_int)
    return finished_boards

> TODO: Beschreibung von add_board_with_reflection() schreiben.
> TODO: Schauen, ob sie auch für die Rückwärtsgenerierung verwendet werden kann.

In [ ]:
def add_all_to_used(board_int):
    USED_BOARDS.add(board_int)
    if not USER_WANTS_PAWN:
        for reflection in mirror_board_all(board_int):
            USED_BOARDS.add(reflection)
    return {board_int}

> TODO: Beschreibung von store_sn schreiben

In [ ]:
# TODO
def store_sn(sn):
    # for s in sn:
    #     display(to_board(s, PIECE_LIST))
    pass

> Momentaner start hier positioniert, um zu zeigen, dass s0 funktioniert
> Bemerkung: Es müssen alle Zellen davor ausgeführt werden, dass es funktioniert! (Genauer: Es muss USED_BOARDS resetted werden!)

In [ ]:
start_calculation()

In [ ]:
def calculate_next_sn(sn):
    sn_p_1 = set()
    for board_int in sn:
        # Nicht sicher, ob hier schon die Bretterstellung notwendig ist (glaube nicht)
        board_rep = to_board(board_int)
        board_rep.turn = board_rep.turn ^ True

        # Berücksichtigung bei dieser Trennung, dass irgendwann auch der Pawn in eine Queen umgewandelt wird und dann muss USER_WANTS_PAWN = False sein -> Danach aber vorsichtig bei Spiegelungen sein, weil diese auch nicht instant verwendet werden können -> Möglicherweise doch so umsetzbar
        if USER_WANTS_PAWN:
            do_pawn_stuff(board_int)
        else:
            do_not_pawn_stuff(board_int)

        # wenn nicht notwendig, muss das hier auch weg und in den Funktionen muss das Turn-Gedrehe umgesetzt werden!
        board_rep.turn = board_rep.turn ^ True

    return sn_p_1
    pass

> TODO: Diese Funktion wird verwendet, um jeglichen Stuff bzgl. Pawns abzuarbeiten -> Modularisierung von Funktionen und Trennung der Funktionalität

# Beriech für die Implementierung von nicht-Bauern stuff

In [ ]:
def do_not_pawn_stuff(board_int):
    board_rep = to_board(board_int)
    board_rep.turn = board_rep.turn ^ True
    sn_p_1 = set()
    for pLMove in board_rep.pseudo_legal_moves:
        # Backwards-execuion of Move
        board_rep.push(pLMove)
        board_rep.turn = board_rep.turn ^ True

        # Check if Move is valid -> Kann man Auslagern? Unsicher wegen continue? Maximal, dass man in der Funktion ein Bool weitergibt und anhand davon etwas macht
        if not chess_board.is_valid() or chess_board.outcome() is not None:
            chess_board.turn = chess_board.turn ^ True
            chess_board.pop()
            continue

        new_board_int = to_integer(board_rep, PIECE_LIST)

        # Erstellung der Mirrored Boards und das Hinzufügen in Uniques passiert erst in check_black_determinism bzw. mal schauen, wo es genau passiert
        if new_board_int not in USED_BOARDS and new_board_int not in sn_p_1:
            sn_p_1.add(new_board_int)

    return sn_p_1

# Bereich für alle Bauern-Funktionen

In [ ]:
def do_pawn_stuff(board_int):
    # Frage: Brauche ich pawn_positions überhaupt, wenn ich das so trenne, in alter Version werden sie nur zur Überprüfung verwendet, ob Pawns auf dem Feld sind (wissen wir durch USER_WANTS_PAWN) und ob pseudo_legal_moves Pawns enthalten (da wir dieses Szenario woanders behandeln (ohne Pawns) ist das irrelevant;

    # Bezüglich der Trennung mit dem anderen shit, wenn eine Queen auf dem Brett steht, muss hier eine Funktion definiert werden, die die Züge der Queen rückwärts darstellt! Glaube die Position von do_queen_backwards sollte passen()

    board_rep = to_board(board_int)
    pawn_positions = find_pawn(board_rep)
    board_rep.turn = board_rep.turn ^ True
    if board_rep.turn:
        # Es stehe Pawns auf dem Schachbrett
        if len(pawn_positions) > 0:
            pawn_moves()
        # Stehen keine Pawns, also irgendwelche anderen Figuren, die mal ein Pawn waren, auf dem Schachbrett, da Queen optimal, Queens
        else:
            queen_positions = check_top_row_for_queen()
            if queen_positions:
                replace_queen(board_rep, queen_positions)
            else:
                do_queen_backwards()
    pass

> TODO: Beschreibung von find_pawn():

In [ ]:
def find_pawn(board_object):
    # Anmerkung: Logisch gesehen kann es nur einen Bauern geben (wegen Bedingung). Deswegen muss eigentlich nicht über Pieces iteriert werden, sondern es kann direkt zurückgegeben werden und dann geschaut, ob es nicht None ist (Falls das mit dem Erstellen von einem SquareSet übereinstimmt)
    positions = set()
    for p_position in board_object.pieces(chess.PAWN, chess.WHITE):
        positions.add(chess.square_name(p_position))
    return positions

In [ ]:
def pawn_moves():
    # Müssen auch Anfangszüge berücksichtigt werden? Also Doppelzug? Wenn ja, dann pawn_moves, wenn nein, dann pawn_move
    pass

In [ ]:
def check_top_row_for_queen():
    # Kann man das nicht ebenfalls mit der Funktion board.pieces(chess.QUEEN, chess.WHITE) lösen? Danach einfach noch schauen, ob der Wert in range(56,64) liegt -> Müssen dann nicht darüber iterieren.
    # Und bei Check erwarte ich implizit erstmal ein True oder False, würde zusätzlich noch ein get definieren, wenn auch wirklich der Wert davon gefragt ist
    pass

In [ ]:
def replace_queen(board_rep, queen_positions):
    # Same case as find_pawn, es kann doch nur eine toprow_queen_position geben
    pass

In [ ]:
def do_queen_backwards():
    # könnte auch im Sinne von dem not_pawn_stuff implementiert werden, wenn die Züge rückwärts mit der Spiegelung berechnet werden. Man kann zusätzlich ein Flag mitgeben, das für die Queen die Spiegelungen ausstellt (Idee, nicht muss)
    pass

> TODO: Funktion, die zur Generierung von anderen Zügen ohne Bauern benötigt wird.

* Funktion zur Überprüfung der Züge von schwarz (dass diese auch bei einer Menge m landen mit m < n)
    * Einfach überprüfen, ob Züge in used_boards landen

In [ ]:
def check_black_moves(uniques):
    # for int_board in uniques:

    # am Ende muss hier add_all_to_used für die übrigen boards aufgerufen werden!
    pass